In [1]:
import tensorflow as tf
from utils import save 
import numpy as np
from models._12_net import calib_12Net
from utils.utilities import batch_selection
import sys
from utils.params import *
import time
from utils import save

In [2]:
def valid_correct_percentage(pred, truth, top_numb=3):
    assert(truth.shape[0] == pred.shape[0])
    top_score = [arr.argsort()[-top_numb:][::-1] for arr in pred]
    score_list = [1 if truth[i] in top_score[i] else 0 for i in range(truth.shape[0])]
    percentage = np.sum(score_list) / truth.shape[0]
    return percentage

In [14]:
batch_size = 256
epochs = 2000
lr = [1e-3, 7e-4, 4e-4]
bias_init = [0.0]
reg = [0.0, 1e-4]
_12_calib_batch = batch_selection()

In [4]:
calibrate_db_12 = save.loader('calibrate_db_12_long.txt')
calibrate_db_12 = calibrate_db_12[:int(len(calibrate_db_12) / batch_size) * batch_size]
inputs_calib_12 = np.zeros((len(calibrate_db_12), 12, 12, 3))
targets_calib_12 = np.zeros((len(calibrate_db_12), 45), dtype=np.int32)

for i in range(len(calibrate_db_12)):
    inputs_calib_12[i,:] = calibrate_db_12[i][0]
    targets_calib_12[i,calibrate_db_12[i][1]] = 1

Loaded from D:\programing project\python project\trying stuffpickle/


In [5]:
inputs = np.zeros((batch_size, 12, 12, 3))
targets = np.zeros((batch_size, 45), dtype=np.int32)

In [6]:
g=tf.Graph()
with g.as_default():
    input_12_node = tf.placeholder(tf.float32, [None, 12, 12, 3])
    target_12_node = tf.placeholder(tf.float32, [None, 45])
    keep_prob = tf.placeholder(tf.float32)
    _calib_12 = calib_12Net(input_12_node, target_12_node, keep_prob, 0.0, 0.0)

sess = tf.InteractiveSession(config=tf.ConfigProto(
  allow_soft_placement=True, log_device_placement=True), graph=g)
sess.run(tf.global_variables_initializer())

In [7]:
result_list = save.loader('calibrate_db_12_test.txt')
valid_input = np.concatenate([result_list[i][0][np.newaxis] for i in range(len(result_list))])
valid_target = np.asarray([result_list[i][1] for i in range(len(result_list))])


Loaded from D:\programing project\python project\trying stuffpickle/


In [15]:
for r in reg:
    for b in bias_init:
        for l in lr:
            record_l = l
            print('**Bias_init:{}   Learning_rate:{}   Regularization:{}**'.format(b,l,r))

            max_i = 0
            correct_pnt_top3 = 0
            correct_pnt_top1 = 0
            correct_pnt_top3_max = 0
            correct_pnt_top1_max = 0
            bad_cnt = 0

            start = time.time()
            g=tf.Graph()
            with g.as_default():
                input_12_node = tf.placeholder(tf.float32, [None, 12, 12, 3])
                target_12_node = tf.placeholder(tf.float32, [None, 45])
                keep_prob = tf.placeholder(tf.float32)
                _calib_12 = calib_12Net(input_12_node, target_12_node, keep_prob, l, b, r)

            sess = tf.InteractiveSession(config=tf.ConfigProto(
              allow_soft_placement=True, log_device_placement=True), graph=g)
            sess.run(tf.global_variables_initializer())
            #train_writer = tf.summary.FileWriter('./logs/train/Calib_12/bias_init={},lr={}'.format(b,l))

            for i in range(epochs):

                if (i+1) % 5 == 0:
                    pred_target = _calib_12.prediction.eval(feed_dict={input_12_node:valid_input,keep_prob:1.0})
                    correct_pnt_top3 = valid_correct_percentage(pred_target, valid_target, 3)
                    correct_pnt_top1 = valid_correct_percentage(pred_target, valid_target, 1)

                    if correct_pnt_top3 >= correct_pnt_top3_max:
                        max_i = i + 1
                        correct_pnt_top3_max = correct_pnt_top3
                        correct_pnt_top1_max = correct_pnt_top1
                        bad_cnt = 0
                        saver = tf.train.Saver()
                        saver.save(sess, p_model_dir + 'calib_12/max/_Net12_b{}_lr{}_reg{}_drop{}.ckpt'.format(b,record_l,r,0.5))

                    else:
                        bad_cnt += 1

                if bad_cnt % 5 == 0:
                    l = l * .98
                    
                if bad_cnt >= 120:
                    break

                for batch_i in range(int(len(calibrate_db_12) / batch_size)):
                    iters = i*int(len(calibrate_db_12) / batch_size) + batch_i
                    inputs, targets = _12_calib_batch.next_batch(inputs_calib_12, targets_calib_12, batch_size)
                    feed_train = {
                        input_12_node:inputs,
                        target_12_node:targets,
                        keep_prob:.5
                    }
                    feed_loss = {
                        input_12_node:inputs,
                        target_12_node:targets,
                        keep_prob:1.0

                    }

                    cost = _calib_12.loss.eval(feed_loss)
                    sys.stdout.write('\rEpoch:' + str(i+1) + \
                                     '    Iters:' + str(iters+1) + '/' + str(int(len(calibrate_db_12) / batch_size)*epochs) + \
                                     '    Loss:' + str(cost)[:8] + \
                                     '    Cur_Accuracy:' + str(correct_pnt_top1*100)[:5] + '%/' + str(correct_pnt_top3*100)[:5] + '%' + \
                                     '    Max_Acc/Epo:' + str(correct_pnt_top1_max*100)[:5] + '%/' + str(correct_pnt_top3_max*100)[:5] + \
                                                          '%/' + str(max_i) + \
                                     '    Unimprove_Count:' + str(bad_cnt))
                    #if (iters + 1) % 10 == 0 or iters == 0:
                        #summary = _calib_12.merged.eval(feed_loss)
                        #train_writer.add_summary(summary, iters)

                    _calib_12.train_step.run(feed_train)
            end = time.time()
            print('\nTrained Time:{} mins\n'.format(str((end-start)/60.0)[:5]))
            #saver = tf.train.Saver()
            #saver.save(sess, p_model_dir + 'calib_12/_Net12_b{}_lr{}drop{}.ckpt'.format(b,l,0.0))

**Bias_init:0.0   Learning_rate:0.001   Regularization:0.0**
Epoch:834    Iters:65052/156000    Loss:1.15456    Cur_Accuracy:49.55%/77.33%    Max_Acc/Epo:52.88%/78.55%/235    Unimprove_Count:119
Trained Time:9.333 mins

**Bias_init:0.0   Learning_rate:0.0007   Regularization:0.0**
Epoch:789    Iters:61542/156000    Loss:0.69467    Cur_Accuracy:51.11%/77.55%    Max_Acc/Epo:56.22%/82.44%/190    Unimprove_Count:119
Trained Time:9.140 mins

**Bias_init:0.0   Learning_rate:0.0004   Regularization:0.0**
Epoch:714    Iters:55692/156000    Loss:0.491528    Cur_Accuracy:55.11%/78.88%    Max_Acc/Epo:60.0%/82.11%/115    Unimprove_Count:119
Trained Time:7.898 mins

**Bias_init:0.0   Learning_rate:0.001   Regularization:0.0001**
Epoch:1129    Iters:88062/156000    Loss:1.51238    Cur_Accuracy:49.88%/78.0%    Max_Acc/Epo:48.55%/79.33%/530    Unimprove_Count:119
Trained Time:12.96 mins

**Bias_init:0.0   Learning_rate:0.0007   Regularization:0.0001**
Epoch:954    Iters:74412/156000    Loss:1.11557   

In [7]:
for b in bias_init:
    for l in lr:
        print('**Bias_init:{}   Learning_rate:{}**'.format(b,l))
        start = time.time()
        g=tf.Graph()
        with g.as_default():
            input_12_node = tf.placeholder(tf.float32, [None, 12, 12, 3])
            target_12_node = tf.placeholder(tf.float32, [None, 45])
            keep_prob = tf.placeholder(tf.float32)
            _calib_12 = calib_12Net(input_12_node, target_12_node, keep_prob, b, l)
        
        sess = tf.InteractiveSession(config=tf.ConfigProto(
          allow_soft_placement=True, log_device_placement=True), graph=g)
        sess.run(tf.global_variables_initializer())
        train_writer = tf.summary.FileWriter('./logs/train/Calib_12/bias_init={},lr={}'.format(b,l))
        
        for i in range(epochs):
            for batch_i in range(int(len(calibrate_db_12) / batch_size)):
                iters = i*int(len(calibrate_db_12) / batch_size) + batch_i
                inputs, targets = _12_calib_batch.next_batch(inputs_calib_12, targets_calib_12, batch_size)
                feed_train = {
                    input_12_node:inputs,
                    target_12_node:targets,
                    keep_prob:0.8
                }
                feed_loss = {
                    input_12_node:inputs,
                    target_12_node:targets,
                    keep_prob:1.0
                    
                }
                
                cost = _calib_12.loss.eval(feed_loss)
                sys.stdout.write('\rEpoch:' + str(i+1) + \
                                 '    Iters:' + str(iters+1) + '/' + str(int(len(calibrate_db_12) / batch_size)*epochs) + \
                                 '    Loss:' + str(cost)[:8])
                if (iters + 1) % 10 == 0 or iters == 0:
                    summary = _calib_12.merged.eval(feed_loss)
                    train_writer.add_summary(summary, iters)
                
                _calib_12.train_step.run(feed_train)
        end = time.time()
        print('    Trained Time:{} mins\n'.format(str((end-start)/60.0)[:5]))
        saver = tf.train.Saver()
        saver.save(sess, p_model_dir + 'calib_12/_Net12_b{}_lr{}drop{}.ckpt'.format(b,l,0.0))

**Bias_init:0.0   Learning_rate:0.01**
Epoch:600    Iters:11400/11400    Loss:0.348112    Trained Time:1.873 mins

**Bias_init:0.0   Learning_rate:0.005**
Epoch:600    Iters:11400/11400    Loss:1.36653    Trained Time:1.885 mins

**Bias_init:0.0   Learning_rate:0.001**
Epoch:600    Iters:11400/11400    Loss:0.01272    Trained Time:1.883 mins

**Bias_init:0.01   Learning_rate:0.01**
Epoch:600    Iters:11400/11400    Loss:0.044465    Trained Time:1.876 mins

**Bias_init:0.01   Learning_rate:0.005**
Epoch:600    Iters:11400/11400    Loss:0.991462    Trained Time:1.884 mins

**Bias_init:0.01   Learning_rate:0.001**
Epoch:600    Iters:11400/11400    Loss:0.079620    Trained Time:1.889 mins

**Bias_init:0.1   Learning_rate:0.01**
Epoch:600    Iters:11400/11400    Loss:0.409649    Trained Time:1.874 mins

**Bias_init:0.1   Learning_rate:0.005**
Epoch:600    Iters:11400/11400    Loss:0.130105    Trained Time:1.883 mins

**Bias_init:0.1   Learning_rate:0.001**
Epoch:600    Iters:11400/11400    

In [7]:
for i in range(epochs):
    summary = _calib_12.merged.eval(feed)
    train_writer.add_summary(summary, i)
    if (i + 1) % 10 == 0 or i == 0:
        cost = _calib_12.loss.eval(feed)
        print('Epochs:{}'.format(str(i)), 'Loss:{}'.format(str(cost)))
    _calib_12.train_step.run(feed)

Epochs:0 Loss:3.80697
Epochs:9 Loss:2.54898
Epochs:19 Loss:2.36249
Epochs:29 Loss:2.33224
Epochs:39 Loss:2.31482
Epochs:49 Loss:2.30627
Epochs:59 Loss:2.30354
Epochs:69 Loss:2.30283
Epochs:79 Loss:2.3027
Epochs:89 Loss:2.30267
Epochs:99 Loss:2.30263


In [44]:
calibrate_db_12_inputs = np.concatenate([calibrate_db_12[i][0][np.newaxis] for i in range(len(calibrate_db_12))])
calibrate_db_12_targets = np.array([calibrate_db_12[i][1] for i in range(len(calibrate_db_12))])

In [103]:
b_init = 0.0
lr = 5e-2
w_std = 0.1

In [104]:
def weight_variable(shape, scale, name=None):
    init = tf.truncated_normal(shape=shape, stddev=w_std) / scale
    return tf.Variable(init, name=name)

def weight_variable_refine(shape, scale, name=None):
    init = tf.random_normal(shape=shape) / scale
    return tf.Variable(init, name=name)

def bias_variable(shape, name=None):
    init = tf.constant(value=b_init, shape=shape)
    return tf.Variable(init, name=name)

def conv2d(x, W, stride, pad = 'SAME'):
    return tf.nn.conv2d(x, W, strides=[1,stride,stride,1], padding=pad)

def max_pool(x, kersize, stride, pad = 'SAME'):
    return tf.nn.max_pool(x, ksize=[1,kersize,kersize,1], strides=[1,stride,stride,1], padding=pad)

In [105]:
tf.reset_default_graph()
input_12_node = tf.placeholder(tf.float32, [None, 12, 12, 3])
target_12_node = tf.placeholder(tf.float32, [None, 45])

In [106]:
# conv1
w_conv1 = weight_variable([3,3,3,16], tf.sqrt(3*3*3.0/2), 'w1')
b_conv1 = bias_variable([16], 'b1')

h_conv1 = tf.nn.relu(conv2d(input_12_node, w_conv1, 1) + b_conv1)
h_pool1 = max_pool(h_conv1, 3, 2)

# fc1
w_fc1 = weight_variable([6*6*16,128], tf.sqrt(6*6*16.0/2), 'w2')
b_fc1 = bias_variable([128], 'b2')
h_pool1_flat = tf.reshape(h_pool1, [-1, 6*6*16])
h_fc1 = tf.nn.relu(tf.matmul(h_pool1_flat, w_fc1) + b_fc1)

# fc_conv2
w_fc2 = weight_variable([128,45], tf.sqrt(128.0/2), 'w3')
b_fc2 = bias_variable([45], 'b3')
h_fc2 = tf.matmul(h_fc1, w_fc2) + b_fc2

# softmax prediction
prediction = tf.nn.softmax(h_fc2)

# loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=h_fc2, labels=target_12_node))

# optimizer
train_step = tf.train.AdamOptimizer(lr).minimize(loss)

In [107]:
batch_size = 10
epochs = 100

In [108]:
inputs = np.zeros((batch_size, 12, 12, 3))
targets = np.zeros((batch_size, 45), dtype=np.int32)

In [109]:
for i in range(batch_size):
    inputs[i,:] = calibrate_db_12[i][0]
    targets[i,calibrate_db_12[i][1]] = 1

In [110]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())


In [111]:
feed = {
    input_12_node:inputs,
    target_12_node:targets
}

In [100]:
for i in range(epochs):
    if (i + 1) % 10 == 0 or i == 0:
        cost = loss.eval(feed)
        print('Epochs:{}'.format(str(i)), 'Loss:{}'.format(str(cost)))
    train_step.run(feed)

Epochs:0 Loss:3.52852
Epochs:9 Loss:1.5357
Epochs:19 Loss:0.006028
Epochs:29 Loss:6.20592e-05
Epochs:39 Loss:4.56565e-06
Epochs:49 Loss:5.18552e-06
Epochs:59 Loss:4.01731e-06
Epochs:69 Loss:2.55106e-06
Epochs:79 Loss:1.63316e-06
Epochs:89 Loss:1.14441e-06
Epochs:99 Loss:8.70226e-07


In [112]:
for i in range(epochs):
    if (i + 1) % 10 == 0 or i == 0:
        cost = loss.eval(feed)
        print('Epochs:{}'.format(str(i)), 'Loss:{}'.format(str(cost)))
    train_step.run(feed)

Epochs:0 Loss:3.80697
Epochs:9 Loss:2.33776
Epochs:19 Loss:2.30832
Epochs:29 Loss:2.30634
Epochs:39 Loss:2.30348
Epochs:49 Loss:2.3029
Epochs:59 Loss:2.30273
Epochs:69 Loss:2.30267
Epochs:79 Loss:2.30259
Epochs:89 Loss:2.30259
Epochs:99 Loss:2.30259
